In [41]:
import xlwings as xw
import pandas as pd
wb_cy = xw.Book('cytiva.xlsm')
SHEET_NAMES =  ['Temp_DB', 'Shipment information', '인수증', 
'대리점송장', '대리점 출고대기', '로컬리스트', 'In-Transit part report', '기타리스트',
 '출고리스트', 'Cytiva Inventory BIN']
import win32com.client as cli

outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩
inbox = outlook.GetDefaultFolder(6) # 받은편지함
msg = inbox.Items #메세지 정보
msg_counter = msg.count # 총갯수

In [8]:
shape_name = 'SVC_CCH_23011901'
idx = int(shape_name.split('_')[2])
idx

23011901

In [36]:
ws_main = wb_cy.sheets['통합제어']
idx_col = ws_main.range('J12:J500').options(numbers=int).value
ws_main.range('J12').column

10

In [13]:
row_nm = idx_col.count(idx)

In [34]:
chs_rng = ws_main.range(11 +row_nm,17)
current_status = chs_rng.value
current_status

'REQUESTED'

In [27]:
ML_STATUS= ['REQUESTED','PROCESSING', 'SHIPPED']

ML_FOLDERS = ['inbox','1_Requests', '2_Processing', '3_ShipConfirmed']

In [39]:
ml_dict =dict(zip(ML_STATUS,ML_FOLDERS[1:]))
ml_dict[current_status]

'1_Requests'

In [49]:
argu_folder = inbox.Parent.Folders(ml_dict[current_status])

In [50]:
msg = argu_folder.Items
part_request = []
service_request = 'SVC'
for ms in msg:
    if service_request in ms.Subject:
        part_request.append(ms)
# 서비스만 추출하기



In [52]:
part_request[0].Subject

'SVC_CCH_23011901'

In [56]:
def get_mail_status(shape_name=None):
    """
    메일제목을 argument로 받으면 현재상태의 해당 메일의 현재 ML_BIN, STATUS를 반환 
    """
    ws_main= wb_cy.sheets['통합제어']
    
    ml_status = ['REQUESTED','PROCESSING', 'SHIPPED']
    ml_folders = ['inbox','1_Requests', '2_Processing', '3_ShipConfirmed']
    
    ml_dict = dict(zip(ml_status,ml_folders[1:]))
    
    if shape_name != None:
        
        shape_name = shape_name.replace('_prfm','')

        idx = int(shape_name.split('_')[2])
        emp_row_nm = get_empty_row(col='J') ## J = 10, ML_INDEX 컬럼
        idx_list = ws_main.range("J12:J"+str(emp_row_nm+1)).options(numbers=int).value
        row_nm = idx_list.count(idx)
        status_cel = ws_main.range(11 + row_nm,17).value ## Q=17, STATUS 컬럼 , 11 ML_INDEX의 row 번호
        fd_bin = ml_dict[status_cel]
        ms_list = Email.get_email_obj(fd_bin)
    
    return ms_list
        

In [54]:
def get_email_obj(folder_name = 'inbox'):

    outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩

    if folder_name == 'inbox':

        argu_folder = outlook.GetDefaultFolder(6) # 받은편지함
    else:
        argu_folder = outlook.GetDefaultFolder(6).Parent.Folders(folder_name) # 1번폴더


    msg = argu_folder.Items #메세지 정보
    part_request = []
    service_request = 'SVC'
    for ms in msg:
        if service_request in ms.Subject:
            part_request.append(ms)
    # 서비스만 추출하기

    return part_request

In [55]:
def get_empty_row(sheet=wb_cy.selection.sheet,col=1):
    """
    특정컬럼의 마지막 값의 행번호 구하기
    """
    sel_sht = sheet
    col_num = col
    if type(col) == int :
        row_start_nm = sel_sht.range(1048576,col_num).end('up').row + 1 
    elif type(col) == str :
        row_start_nm = sel_sht.range(col+str(1048576)).end('up').row + 1 
    return row_start_nm

In [ ]:
get_mail_status()

In [59]:
# 제목입력 받으면 oracle도 조회
import cx_Oracle
import os

In [111]:
# LOCATION = r"C:\Users\lms46\Desktop\fulfill\instantclient_fulfill"
# os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
# cx_Oracle.init_oracle_client(lib_dir=LOCATION)

connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()

In [61]:
ml_sub = 'SVC_CCH_23011901'

In [90]:
ml_status_db = cursor.execute(f'select * from MAIL_STATUS where ML_SUB = (:name1)',name1=ml_sub)

In [91]:
for ml in ml_status_db:
    df_status = pd.DataFrame(ml)

In [104]:
df_status.T[[1,2,3,4]]

,1,2,3,4
0,SVC_CCH_23011901,REQUESTED,2023-01-23 21:50:30,1_Requests


In [ ]:
argu = 'SVC_CCH_23011901'

In [98]:
def DataWarehouse():
    try:
        os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
        cx_Oracle.init_oracle_client(lib_dir=LOCATION)
        connection = cx_Oracle.connect(
            user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

        )
        cursor = connection.cursor()
        return cursor

    except:

        connection = cx_Oracle.connect(
            user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

        )
        cursor = connection.cursor()
        return cursor

In [177]:
def get_mail_status(ml_sub):
    """
    메일제목을 argument로 받으면 현재상태의 해당 메일의 현재 ML_BIN, STATUS를 반환 
    """
    ml_sub = ml_sub.replace('_prfm','')
    cur = DataWarehouse()
    query = 'select * from MAIL_STATUS where ML_SUB = (:name1)'
    db_obj = cur.execute(query, name1= ml_sub)
    df_status = pd.DataFrame(db_obj.fetchall())
    # 최신 날짜로 업데이트된 부분만 가져오기
    df_status = df_status.sort_values(3,ascending=False).iloc[[0]][[1,2,3,4]]
    return df_status


In [179]:
get_mail_status(ml_sub)

,1,2,3,4
0,SVC_CCH_23011901,REQUESTED,2023-01-23 21:50:30,1_Requests


In [178]:
msg = get_email_obj(get_mail_status(ml_sub)[4][0])

find_ms = []
for ms in msg :
    if ms.Subject == ml_sub:
        find_ms.append(ms)
find_ms[0].Subject

'SVC_CCH_23011901'

In [133]:
df_status = get_mail_status()
df_status

""


In [130]:
df_status[3] = pd.to_datetime(df_status[3], format='%Y-%m-%d %H:%M:%S', errors='raise')

In [131]:
df_status[3]

0   2023-01-23 21:50:30
Name: 3, dtype: datetime64[ns]

In [117]:
tmp_rows = cursor.execute('select * from MAIL_STATUS')
rows_tmp = []
for row in tmp_rows:
    rows_tmp.append(row)
pd.DataFrame(rows_tmp)

,0,1,2,3,4
0,7,SVC_CCH_23011901,REQUESTED,2023-01-23 21:50:30,1_Requests
1,8,SVC_LMR_23011902,REQUESTED,2023-01-23 21:50:30,1_Requests
2,9,SVC_LWW_23011903,REQUESTED,2023-01-23 21:50:30,1_Requests


In [174]:
tmp_rows = cursor.execute('select * from MAIL_STATUS')
df_status = pd.DataFrame(tmp_rows.fetchall())
df_status[3] = pd.to_datetime(df_status[3], format='%Y-%m-%d %H:%M:%S', errors='raise')

In [175]:
df_status = df_status.sort_values(3,ascending=False).iloc[[0]]

In [176]:
df_status

,0,1,2,3,4
0,7,SVC_CCH_23011901,REQUESTED,2023-01-23 21:50:30,1_Requests
